<a href="https://colab.research.google.com/github/cgoodpaster/NMA/blob/main/Clean_Cluster_Buster_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##House-keeping


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.decomposition import PCA
import pickle
import os

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Downloads all Needed Files from Drive

In [3]:
%%capture
!pip install gdown

dest_folder = '/content/drive/MyDrive/HCP_steinmetz/'
if not os.path.exists(dest_folder):
  os.makedirs(dest_folder)  # create folder if it does not exist

os.chdir(dest_folder)


!gdown '1296ESQEAYK7dSiwX5szspI635eQu4uNt' --remaining-ok 

with open("/content/drive/MyDrive/HCP_steinmetz/all_firing_rate.pickle", "rb") as file:
    all_firing_rate = pickle.load(file)
#The file has all the firing rates, for all the mouse, for all the sessions, for all the neurons. You can plot the fire rate for one neuron: 

!gdown '1zwsRy9Tlb0L1jOyL8Uqm5O_C8NnjS3Qo' --remaining-ok
with open("/content/drive/MyDrive/HCP_steinmetz/alldat_dict.pickle", "rb") as file:
    alldat_dict = pickle.load(file)

##Calculate the mean firing rate for each neuron (in all sessions) for correct and incorrect Trials

In [5]:
all_firing_rate_corr = {}
all_firing_rate_incorr = {}


for mouse in all_firing_rate.keys():
  all_firing_rate_corr[mouse] = {}
  all_firing_rate_incorr[mouse] = {}
  for ses in range(len(alldat_dict[mouse])):
    #initialize np array
    # all_firing_rate_corr[mouse][ses] = np.zeros((all_firing_rate[mouse][ses].shape[0], np.where(alldat_dict[mouse][ses]['feedback_type'] == 1.0)[0].shape[0]  ))
    # all_firing_rate_incorr[mouse][ses] = np.zeros((all_firing_rate[mouse][ses].shape[0], np.where(alldat_dict[mouse][ses]['feedback_type'] == -1.0)[0].shape[0]  ))

    # Storing outuput
    all_firing_rate_corr[mouse][ses]  = np.mean(all_firing_rate[mouse][ses][:,np.where(alldat_dict[mouse][ses]['feedback_type'] == 1.0)[0]], axis=1) #correct
    all_firing_rate_incorr[mouse][ses] = np.mean(all_firing_rate[mouse][ses][:,np.where(alldat_dict[mouse][ses]['feedback_type'] == -1.0)[0]], axis=1) #incorrect

##Combines all neurons into one array

In [6]:
total_neurons_corr = 0
total_neurons_in = 0

total_neurons_corr = []

for mouse in all_firing_rate_corr.keys():
  for ses in range(len(alldat_dict[mouse])):
    total_neurons_corr = np.append(total_neurons_corr, all_firing_rate_corr[mouse][ses])
    

    #print(total_neurons_corr.shape)


total_neurons_in = []
for mouse in all_firing_rate_incorr.keys():
  for ses in range(len(alldat_dict[mouse])):
    total_neurons_in = np.append(total_neurons_in, all_firing_rate_incorr[mouse][ses])
    #print(total_neurons_in.shape)

print(total_neurons_corr.shape,total_neurons_in.shape)

clustering_x = np.vstack((total_neurons_corr, total_neurons_in)).T


(33958,) (33958,)


## Dendrogram Creation

In [ ]:
##Clustering Analysis
import scipy.cluster.hierarchy as sch
dendrogram = sch.dendrogram(sch.linkage(clustering_x, method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()